In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
def reReadData():
  df=pd.read_csv('gdrive/My Drive/Data_Version_4.csv',header=0, index_col=0, 
                infer_datetime_format=True, parse_dates=['Date'])
  df['log_ret'] = np.log(df['Russell3000_Adj_Close']) - np.log(df['Russell3000_Adj_Close'].shift(1))
  #df = df.dropna()
  #df = df[train_start:test_end]
  #df_train, df_test = df[train_start:train_end],df[train_end:test_end]
  return df


def plotProb_train(model):
  bear_regime = 0 if model.params['const[0]']<model.params['const[1]'] else 1

  fig, axes = plt.subplots(2, figsize=(15, 15))

  ax = axes[0]
  ax.plot(df_train.index, df_train['Russell_Close'])
  ax.set(title='Russell_Close')
  
  ax = axes[1]
  ax.plot(df_train.index, model.filtered_marginal_probabilities[bear_regime])
  ax.set(title='Filtered probability of bear_regime')

  plt.show()


def plotProb_test(model):
  bear_regime = 0 if model.params['const[0]']<model.params['const[1]'] else 1

  fig, axes = plt.subplots(2, figsize=(15, 15))

  ax = axes[0]
  ax.plot(df_test.index, df_test['Russell_Close'])
  ax.set(title='Russell_Close')
  
  ax = axes[1]
  ax.plot(df_test.index, model.filtered_marginal_probabilities[bear_regime])
  ax.set(title='Filtered probability of bear_regime')

  plt.show()


def labeling(model,threshold1, threshold2):
  bear_regime = 0 if model.params['const[0]'] < model.params['const[1]'] else 1

  label_filtered = []
  
  for i in model.filtered_marginal_probabilities[bear_regime].values:
    if i >= threshold1:
      app = -1
    elif i >= threshold2:
      app = 0
    else:
      app = 1
    label_filtered.append(app)

  label_filtered= np.array(label_filtered)
  
  return label_filtered


def duration(label):

  duration_bull = []
  duration_static = []
  duration_bear = []

  count,prev,trade = 1,1,0

  for i in label:
    if i == prev:
      count+=1
    else:
      trade+=1
      if prev == 1:
        duration_bull.append(count)
        count = 1
      elif prev == 0:
        duration_static.append(count)
        count = 1
      else:
        duration_bear.append(count)
        count = 1
    prev = i
    
  if prev == 1:
    duration_bull.append(count)
  elif prev == 0:
    duration_static.append(count)
  else:
    duration_bear.append(count)

  duration_bull = pd.DataFrame(duration_bull)
  duration_static = pd.DataFrame(duration_static)
  duration_bear = pd.DataFrame(duration_bear)

  return duration_bull, duration_static, duration_bear,trade


def label_train_logret(label):
  return (df_train['log_ret'] * label).sum()

def label_test_logret(label):
  return (df_test['log_ret'] * label).sum()



def Search_train(model, duration_static, duration_bear):
  threshold1, threshold2, record =0,0,0
  for i in range(1,101):
    for j in range(i):
      label_filtered = labeling(model,i/100, j/100)
      if ( (duration(label_filtered))[1].mean().values >= duration_static ):
        if ( (duration(label_filtered))[2].mean().values >= duration_bear ) :
          if ( (df_train['log_ret'] * label_filtered).sum() >= record ):
            threshold1, threshold2, record = i,j,(df_train['log_ret'] * label_filtered).sum()
  return threshold1/100, threshold2/100

In [4]:
train_start = '2000-01-01'
train_end = '2017-12-31'
test_end = '2022-01-25'

In [5]:
#df,df_train,df_test = reReadData()
df = reReadData()
print(df)

             PX_LAST     PX_VOLUME  ...  Russell3000_Volume   log_ret
Date                                ...                              
1990-01-02    585.41           NaN  ...                 0.0       NaN
1990-01-03    585.05           NaN  ...                 0.0 -0.000999
1990-01-04    580.97           NaN  ...                 0.0 -0.006316
1990-01-05    575.74           NaN  ...                 0.0 -0.010058
1990-01-08    577.78           NaN  ...                 0.0  0.003448
...              ...           ...  ...                 ...       ...
2022-01-19  14547.22  1.322548e+09  ...                 0.0 -0.010180
2022-01-20  14378.11  1.355285e+09  ...                 0.0 -0.011763
2022-01-21  14099.17  1.917342e+09  ...                 0.0 -0.019599
2022-01-24  14173.27  2.178968e+09  ...                 0.0  0.000000
2022-01-25  13982.41  1.611222e+09  ...                 0.0  0.000000

[8090 rows x 45 columns]


In [6]:
for k in [5,10,15]:
  name = 'log_ret_MA'+str(k)
  df[name] = df['log_ret'].rolling(k).mean()
  name1 = 'log_ret_EWMA'+str(k)
  df[name1] = df['log_ret'].ewm(span=k).mean()  
df

,PX_LAST,PX_VOLUME,T1M,T3M,T6M,T1Y,T3Y,T5Y,T10Y,T30Y,T10Y3M,T10Y2Y,LIBOR_3M,TEDRATE,InterestRate,WTI_Futures_Price,WTI_Futures_Return,WTI_Spot_Price,WTI_Spot_Change,Commdity_channel_index_13,Williams_pct_14,RSI_14,Parabolic_0.02_0.2_0.02,DMI_14_ADX,Fear_Greedy_Index,CPI_monthly percent change,CPI_Release,Unemployment_rate,Unemployment_Release,change_in_total_nonfarm_thousand,Nonfarm_Release,VIX_Close,GDP_QOQ_change,GDP_Release,SPY_Adj_Close,SPY_Volume,SP_Adj_Close,SP_Volume,Mkt-RF,SMB,HML,RF,Russell3000_Adj_Close,Russell3000_Volume,log_ret,log_ret_MA5,log_ret_EWMA5,log_ret_MA10,log_ret_EWMA10,log_ret_MA15,log_ret_EWMA15
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-01-02,585.41,NaN,NaN,7.83,7.50,7.81,7.90,7.87,7.94,8.00,0.11,0.07,8.38000,0.80,8.54,22.89,0.0490,22.89,0.0490,158.6294,-0.8405,69.0585,190.157,17.2194,2.3009,0.3,0.0,5.4,0.0,276.0,0.0,17.240000,0.8,0.0,NaN,NaN,359.69,1.620700e+08,1.44,-0.68,-0.07,0.026,200.309998,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,585.05,NaN,NaN,7.89,7.53,7.85,7.96,7.92,7.99,8.04,0.10,0.05,8.38000,0.75,8.37,23.68,0.0345,23.68,0.0345,166.5505,-6.7745,68.4778,190.767,18.4101,3.0652,0.3,0.0,5.4,0.0,276.0,0.0,18.190001,0.8,0.0,NaN,NaN,358.76,1.923300e+08,-0.06,0.74,-0.26,0.026,200.110001,0.0,-0.000999,NaN,-0.000999,NaN,-0.000999,NaN,-0.000999
1990-01-04,580.97,NaN,NaN,7.84,7.50,7.82,7.93,7.91,7.98,8.04,0.14,0.06,8.38000,0.79,8.29,23.41,-0.0114,23.41,-0.0114,104.2355,-19.9820,61.8875,191.575,18.1196,1.8271,0.3,0.0,5.4,0.0,276.0,0.0,19.219999,0.8,0.0,NaN,NaN,355.67,1.770000e+08,-0.71,0.42,-0.22,0.026,198.850006,0.0,-0.006316,NaN,-0.004189,NaN,-0.003924,NaN,-0.003835
1990-01-05,575.74,NaN,NaN,7.79,7.46,7.79,7.94,7.92,7.99,8.06,0.20,0.09,8.38000,0.84,8.20,23.08,-0.0141,23.08,-0.0141,56.8323,-36.1185,54.9317,192.319,17.4400,-0.2747,0.3,0.0,5.4,0.0,84.0,1.0,20.110001,0.8,0.0,NaN,NaN,352.20,1.585300e+08,-0.85,0.77,-0.21,0.026,196.860001,0.0,-0.010058,NaN,-0.006969,NaN,-0.006390,NaN,-0.006192
1990-01-08,577.78,NaN,NaN,7.79,7.49,7.81,7.95,7.92,8.02,8.09,0.23,0.12,8.38000,0.84,8.27,21.62,-0.0633,21.62,-0.0633,43.9596,-30.0539,56.8927,193.003,16.6146,-0.5619,0.3,0.0,5.4,0.0,84.0,0.0,20.260000,0.8,0.0,NaN,NaN,353.79,1.401100e+08,0.30,-0.41,-0.26,0.026,197.539993,0.0,0.003448,NaN,-0.002642,NaN,-0.003148,NaN,-0.003280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,14547.22,1.322548e+09,0.05,0.17,0.36,0.57,1.33,1.62,1.83,2.14,1.66,0.79,0.25514,0.09,0.08,86.96,0.0179,85.58,-0.0135,-167.3056,-99.3827,32.9727,2780.178,21.6253,-124.8524,0.5,0.0,3.9,0.0,199.0,0.0,23.850000,2.3,0.0,451.750000,109357600.0,4532.76,3.387370e+09,-0.31,0.06,0.53,0.000,2633.860107,0.0,-0.010180,-0.008528,-0.008960,-0.006132,-0.006185,-0.004039,-0.004643
2022-01-20,14378.11,1.355285e+09,0.05,0.17,0.36,0.60,1.34,1.62,1.83,2.14,1.66,0.75,0.25886,0.08,0.08,86.90,-0.0007,84.67,-0.0106,-163.7336,-98.8126,29.7787,2768.382,23.3995,-171.7405,0.5,0.0,3.9,0.0,199.0,0.0,25.590000,2.3,0.0,446.750000,122379700.0,4482.73,3.473060e+09,-0.31,0.06,0.53,0.000,2603.060059,0.0,-0.011763,-0.011175,-0.009894,-0.005103,-0.007199,-0.004907,-0.005533
2022-01-21,14099.17,1.917342e+09,0.05,0.17,0.35,0.58,1.28,1.54,1.75,2.07,1.58,0.74,0.25771,0.09,0.08,85.14,-0.0203,85.43,0.0090,-191.9934,-99.7294,25.4267,2751.593,25.6026,-213.5388,0.5,0.0,3.9,0.0,199.0,0.0,28.850000,2.3,0.0,437.980011,202271200.0,4397.94,3.945810e+09,-0.31,0.06,0.53,0.000,2552.540039,0.0,-0.019599,-0.012187,-0.013129,-0.007058,-0.009453,-0.006071,-0.007291


In [7]:
df = df.iloc[15: , :]

In [8]:
#df = df.dropna()
df = df[train_start:test_end]
df_train, df_test = df[train_start:train_end],df[train_end:test_end]
print(df,df_train,df_test)

             PX_LAST     PX_VOLUME  ...  log_ret_MA15  log_ret_EWMA15
Date                                ...                              
2000-01-03   2909.89  1.748611e+09  ...      0.001836        0.001467
2000-01-04   2795.36  1.805332e+09  ...     -0.000677       -0.003646
2000-01-05   2802.51  2.100967e+09  ...      0.000168       -0.002977
2000-01-06   2784.07  1.972651e+09  ...     -0.000207       -0.002664
2000-01-07   2878.11  2.151953e+09  ...      0.001289        0.001013
...              ...           ...  ...           ...             ...
2022-01-19  14547.22  1.322548e+09  ...     -0.004039       -0.004643
2022-01-20  14378.11  1.355285e+09  ...     -0.004907       -0.005533
2022-01-21  14099.17  1.917342e+09  ...     -0.006071       -0.007291
2022-01-24  14173.27  2.178968e+09  ...     -0.005886       -0.006380
2022-01-25  13982.41  1.611222e+09  ...     -0.006286       -0.005582

[5558 rows x 51 columns]             PX_LAST     PX_VOLUME  ...  log_ret_MA15  log_ret_EW

In [9]:
from statsmodels.tsa.stattools import adfuller
adfuller(df_train['Russell3000_Adj_Close'])

(1.190895094144477,
 0.9959223692556765,
 18,
 4515,
 {'1%': -3.431799174247188,
  '10%': -2.567110868692889,
  '5%': -2.8621803631732403},
 32305.791435624727)

In [10]:
adfuller(df_train['log_ret'])

(-15.310939694230568,
 4.187362600147997e-28,
 20,
 4513,
 {'1%': -3.4317998168351687,
  '10%': -2.5671110198147815,
  '5%': -2.8621806470517406},
 -26857.584095841747)

In [11]:
adfuller(df_train['log_ret_EWMA5'])

(-13.212160923173911,
 1.0417491651649047e-24,
 21,
 4512,
 {'1%': -3.4318001383429073,
  '10%': -2.5671110954259877,
  '5%': -2.862180789085396},
 -36746.45075161783)

In [12]:
duration_static, duration_bear = 0, 0

## Basic HMM with log return

In [13]:
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret'], k_regimes=2,trend='c',switching_variance=False)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret'], k_regimes=2,trend='c',switching_variance=True)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:                log_ret   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               13682.360
Date:                Mon, 14 Feb 2022   AIC                         -27354.719
Time:                        21:45:00   BIC                         -27322.622
Sample:                             0   HQIC                        -27343.414
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0440      0.003    -16.123      0.000      -0.049      -0.039
                             Regime 1 parameters    

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:                log_ret   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               14373.200
Date:                Mon, 14 Feb 2022   AIC                         -28734.399
Time:                        21:45:05   BIC                         -28695.883
Sample:                             0   HQIC                        -28720.833
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0010      0.001     -1.922      0.055      -0.002    2.07e-05
sigma2         0.0004   1.87e-05     20.594      0.0

In [14]:
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret'], k_regimes=3,trend='c',switching_variance=False)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret'], k_regimes=3,trend='c',switching_variance=True)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:                log_ret   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               13960.147
Date:                Mon, 14 Feb 2022   AIC                         -27900.293
Time:                        21:45:16   BIC                         -27836.100
Sample:                             0   HQIC                        -27877.683
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0353      0.002    -20.659      0.000      -0.039      -0.032
                             Regime 1 parameters    

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have bee

                        Markov Switching Model Results                        
Dep. Variable:                log_ret   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               14406.921
Date:                Mon, 14 Feb 2022   AIC                         -28789.842
Time:                        21:45:24   BIC                         -28712.809
Sample:                             0   HQIC                        -28762.709
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0007      0.001     -1.105      0.269      -0.002       0.001
sigma2         0.0004   2.31e-05     18.357      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


In [15]:
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret'], k_regimes=4,trend='c',switching_variance=False)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret'], k_regimes=4,trend='c',switching_variance=True)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:                log_ret   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               14080.430
Date:                Mon, 14 Feb 2022   AIC                         -28126.859
Time:                        21:45:40   BIC                         -28017.730
Sample:                             0   HQIC                        -28088.421
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0281      0.001     20.488      0.000       0.025       0.031
                             Regime 1 parameters    

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-

                        Markov Switching Model Results                        
Dep. Variable:                log_ret   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               14593.152
Date:                Mon, 14 Feb 2022   AIC                         -29146.304
Time:                        21:46:01   BIC                         -29017.917
Sample:                             0   HQIC                        -29101.083
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0009      0.000      6.680      0.000       0.001       0.001
sigma2      3.269e-05   1.58e-06     20.707      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


In [16]:
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret'], k_regimes=5,trend='c',switching_variance=False)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret'], k_regimes=5,trend='c',switching_variance=True)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have bee

                        Markov Switching Model Results                        
Dep. Variable:                log_ret   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               14190.697
Date:                Mon, 14 Feb 2022   AIC                         -28329.393
Time:                        21:46:35   BIC                         -28162.490
Sample:                             0   HQIC                        -28270.605
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0607      0.002    -24.746      0.000      -0.066      -0.056
                             Regime 1 parameters    

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


                        Markov Switching Model Results                        
Dep. Variable:                log_ret   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               14603.693
Date:                Mon, 14 Feb 2022   AIC                         -29147.386
Time:                        21:47:22   BIC                         -28954.805
Sample:                             0   HQIC                        -29079.554
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0063        nan        nan        nan         nan         nan
sigma2      2.795e-05        nan        nan        n

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


## Basic HMM with EWMA5

In [17]:
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret_EWMA5'], k_regimes=2,trend='c',switching_variance=False)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret_EWMA5'], k_regimes=2,trend='c',switching_variance=True)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:          log_ret_EWMA5   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               17999.957
Date:                Mon, 14 Feb 2022   AIC                         -35989.913
Time:                        21:47:25   BIC                         -35957.817
Sample:                             0   HQIC                        -35978.608
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0081      0.000    -22.272      0.000      -0.009      -0.007
                             Regime 1 parameters    

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:          log_ret_EWMA5   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               18452.828
Date:                Mon, 14 Feb 2022   AIC                         -36893.655
Time:                        21:47:27   BIC                         -36855.139
Sample:                             0   HQIC                        -36880.089
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0016      0.000     -7.580      0.000      -0.002      -0.001
sigma2      5.394e-05   2.43e-06     22.198      0.0

In [18]:
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret_EWMA5'], k_regimes=3,trend='c',switching_variance=False)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret_EWMA5'], k_regimes=3,trend='c',switching_variance=True)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:          log_ret_EWMA5   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               18314.894
Date:                Mon, 14 Feb 2022   AIC                         -36609.787
Time:                        21:47:33   BIC                         -36545.594
Sample:                             0   HQIC                        -36587.177
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0209      0.001    -32.840      0.000      -0.022      -0.020
                             Regime 1 parameters    

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:          log_ret_EWMA5   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               18868.263
Date:                Mon, 14 Feb 2022   AIC                         -37712.525
Time:                        21:47:41   BIC                         -37635.493
Sample:                             0   HQIC                        -37685.392
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0021   7.43e-05     27.667      0.000       0.002       0.002
sigma2       5.24e-06   2.36e-07     22.203      0.0

In [19]:
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret_EWMA5'], k_regimes=4,trend='c',switching_variance=False)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret_EWMA5'], k_regimes=4,trend='c',switching_variance=True)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:          log_ret_EWMA5   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               18634.650
Date:                Mon, 14 Feb 2022   AIC                         -37235.300
Time:                        21:47:59   BIC                         -37126.170
Sample:                             0   HQIC                        -37196.861
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0084      0.000     19.203      0.000       0.008       0.009
                             Regime 1 parameters    

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))


In [20]:
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret_EWMA5'], k_regimes=5,trend='c',switching_variance=False)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())
hmm1 = sm.tsa.MarkovRegression(endog=df_train['log_ret_EWMA5'], k_regimes=5,trend='c',switching_variance=True)
hmm1_results = hmm1.fit(search_reps = 100)
print(hmm1_results.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have been re-scaled to continue estimation.
  EstimationWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/regime_switching/markov_switching.py:1478: EstimationWarning: Invalid regime transition probabilities estimated in EM iteration; probabilities have bee

                        Markov Switching Model Results                        
Dep. Variable:          log_ret_EWMA5   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               18667.513
Date:                Mon, 14 Feb 2022   AIC                         -37283.027
Time:                        21:48:57   BIC                         -37116.123
Sample:                             0   HQIC                        -37224.239
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0091      0.000     24.489      0.000       0.008       0.010
                             Regime 1 parameters    

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


                        Markov Switching Model Results                        
Dep. Variable:          log_ret_EWMA5   No. Observations:                 4534
Model:               MarkovRegression   Log Likelihood               19244.543
Date:                Mon, 14 Feb 2022   AIC                         -38429.086
Time:                        21:49:41   BIC                         -38236.506
Sample:                             0   HQIC                        -38361.254
                               - 4534                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0016      0.000     -3.297      0.001      -0.003      -0.001
sigma2         0.0001   6.03e-06     18.019      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
